# Peer-graded Assignment: Построение baseline-решений

В этом задании вам предстоит построить несколько моделей и оценить их качество. Эти модели будут служить нам в качестве baseline-решений и пригодятся сразу для нескольких задач:

Во-первых, на разработку baseline-модели не должно уходить много времени (это требование исходит из оценок затрат на проект в целом - большую часть времени все же нужно потратить на основное решение), процесс должен быть простым, на подавляющем большинстве этапов должны использоваться готовые протестированные инструменты. Все это приводит к тому, что baseline-модели - это дешевый способ сделать грубую оценку потенциально возможного качества модели, при построении которого вероятность допущения ошибок относительно невелика.
Во-вторых, использование моделей разного типа при построении baseline'ов позволяет на раннем этапе сделать предположения о том, какие подходы являются наиболее перспективными и приоритизировать дальнейшие эксперименты.
Наличие baseline-моделей позволяет оценить, какой прирост качества дают различные преобразования, усложнения, оптимизации и прочие активности, которые вы предпринимаете для построения финального решения.
Наконец, если после построение сложного решения оценка его качества будет очень сильно отличаться от оценки качества baseline-моделей, то это будет хорошим поводом поискать в решении ошибки.

### Инструкции

Обучите 3 разные baseline-модели на полученных наборах данных и оцените их качество. На прошлой неделе вы выбрали методику оценки качества моделей на основе кросс-валидации, а также основную и вспомогательные метрики. Оцените с их помощью получившуюся модель. Обратите внимание, что под разными моделями понимаются именно разные алгоритмы классификации. Например, 2 модели, реализующие метод k ближайших соседей с разными k, будут считаться одним baseline-решением (хотя и с разными параметрами). Напоминаем, что отложенная выборка (hold-out dataset) не должна использоваться для построения и оценки baseline-моделей!

Можно (но не обязательно) рассмотреть следующий набор алгоритмов:

1. Линейная модель (например, реализация sklearn.linear_model.RidgeClassifier)
2. Случайный лес (например, реализация sklearn.ensemble.RandomForestClassifier)
3. Градиентный бустинг (например, реализация sklearn.ensemble.GradientBoostingClassifier)

В качестве решения приложите получившийся jupyther notebook. Убедитесь, что в нем присутствуют:

- все baseline-модели, которые вы построили;
- качество всех построенных моделей оценено с помощью кросс-валидации, и это понятно из текста в jupyther notebook;
- все модели оценены с помощью основной и дополнительных метрик качества.

---

__Уважаемые коллеги!__ Если есть возможность, большая 
просьба проверить мою работу по второй неделе, задержался с отправкой.
https://www.coursera.org/learn/data-analysis-project/peer/gAgwL/podghotovka-dannykh-dlia-postroieniia-modieli/review/gEASCC0MEeimRhIQTd6iFA

In [1]:
from sklearn import linear_model as lm
from sklearn import ensemble as es
from sklearn import feature_extraction as fe #.DictVectorizer
from sklearn import cross_validation
from sklearn.model_selection import StratifiedKFold,train_test_split,cross_val_score
from sklearn.metrics import scorer

import pandas as pd
import numpy as np
import seaborn as sb

C:\Users\Anton\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


__Готовим данные__

In [3]:
data = pd.read_csv('orange_small_churn_traintest.csv')

In [4]:
num_columns = list(data.columns[:190])
cat_columns = list(data.columns[190:].drop('Class'))

In [5]:
print data.shape
data.head()

(26800, 231)


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,Class
0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,h2riDjd,LM8l689qOp,NaN,NaN,szEZ,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,BG5Rt2K,NaN,NaN,NaN,PM2D,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,2401.0,0.0,NaN,NaN,NaN,...,NKvz4Af,LM8l689qOp,NaN,kG3k,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,889.0,7.0,NaN,NaN,NaN,...,EXUR6zD,LM8l689qOp,NaN,ELof,453m,RAYp,55YFVY9,NaN,NaN,-1
4,NaN,NaN,NaN,NaN,NaN,1267.0,14.0,NaN,NaN,NaN,...,savS4bQ,LM8l689qOp,NaN,xG3x,TNEC,nIGXDli,F2FyR07IdsN7I,mj86,NaN,-1


__Удалим пустые и константные столбцы, все равно толку от них не будет__

In [6]:
#удаляем пустые столбцы
data.dropna(axis=1, how='all', inplace=True)
#удаляем константные столбцы
const_columns = list()
for x in data.columns:
    if (len(data[x].value_counts())==1):
        const_columns.append(x)

print 'Удаляем столбцы Const', const_columns
data.drop(const_columns, axis=1, inplace=True)   

Удаляем столбцы Const ['Var118', 'Var191', 'Var213', 'Var215', 'Var224']


In [7]:
print data.shape
data.head()

(26800, 208)


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var225,Var226,Var227,Var228,Var229,Class
0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,...,IS7V5Pv,oslk,h2riDjd,LM8l689qOp,NaN,szEZ,RAYp,F2FyR07IdsN7I,NaN,-1
1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Y578iqZ,oslk,BG5Rt2K,NaN,NaN,PM2D,RAYp,F2FyR07IdsN7I,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,2401.0,0.0,NaN,NaN,NaN,...,KtBOw16,oslk,NKvz4Af,LM8l689qOp,kG3k,WqMG,RAYp,F2FyR07IdsN7I,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,889.0,7.0,NaN,NaN,NaN,...,Nvc6w_1,oslk,EXUR6zD,LM8l689qOp,ELof,453m,RAYp,55YFVY9,NaN,-1
4,NaN,NaN,NaN,NaN,NaN,1267.0,14.0,NaN,NaN,NaN,...,r5hLqa2,d0EEeJi,savS4bQ,LM8l689qOp,xG3x,TNEC,nIGXDli,F2FyR07IdsN7I,mj86,-1


In [8]:
#скорректируем коллекции, оставив только те столбцы, которые все еще есть в данных
#отсортируем их, учитывая, что сортировка идет по числам xxx после Var в 'Varxxx'
cat_columns = list(set(cat_columns).intersection(set(data.columns)))
cat_columns.sort(cmp=lambda x, y: 1 if (int(x[3:]) > int(y[3:])) else -1 if (int(x[3:]) < int(y[3:])) else 0)

num_columns = list(set(num_columns).intersection(set(data.columns)))
num_columns.sort(cmp=lambda x, y: 1 if (int(x[3:]) > int(y[3:])) else -1 if (int(x[3:]) < int(y[3:])) else 0)

__Преобразуем категориальные признаки в числовые__

In [9]:
#как мы уже видели раньше, у нас было очень много категориальных столбцов с диким количеством уникальных значений
#попробуем их убрать из обучения
max_uniq_values = 100 #вообще говоря, это тоже подбираемый параметр, но на уровне base-line ограничимся просто разумным значением
cat_uniq_values = data[cat_columns].describe().iloc[1,:].sort_values() 
cat_cut_columns = cat_uniq_values[cat_uniq_values < max_uniq_values].index.tolist()

print cat_cut_columns
print cat_uniq_values

['Var208', 'Var218', 'Var211', 'Var201', 'Var225', 'Var194', 'Var196', 'Var205', 'Var223', 'Var203', 'Var229', 'Var210', 'Var227', 'Var221', 'Var207', 'Var206', 'Var219', 'Var195', 'Var226', 'Var228', 'Var193', 'Var212']
Var208       2
Var218       2
Var211       2
Var201       2
Var225       3
Var194       3
Var196       3
Var205       3
Var223       4
Var203       4
Var229       4
Var210       6
Var227       7
Var221       7
Var207      14
Var206      21
Var219      21
Var195      22
Var226      23
Var228      29
Var193      48
Var212      75
Var204     100
Var197     210
Var192     340
Var216    1536
Var220    3237
Var222    3237
Var198    3237
Var199    3361
Var202    5125
Var217    9933
Var214    9942
Var200    9942
Name: unique, dtype: object


In [10]:
#проверим, есть ли категориальные признаки с пропусками в значениях
cat_hasnan_columns = list()
for x in data[cat_columns]:
    if data[x].isnull().max():
        cat_hasnan_columns.append(x)
print 'Категориальные столбцы, содержащие NaN: %s' % cat_hasnan_columns

Категориальные столбцы, содержащие NaN: ['Var192', 'Var194', 'Var197', 'Var199', 'Var200', 'Var201', 'Var202', 'Var203', 'Var205', 'Var206', 'Var208', 'Var214', 'Var217', 'Var218', 'Var219', 'Var223', 'Var225', 'Var229']


In [11]:
#займемся кодированием признаков, как это было показано в курсе 
#Предобработка данных и логистическая регрессия для задачи бинарной классификации (Course2Week3)

Xcat = data[cat_cut_columns] #cat_columns
Xcat.fillna('NA', inplace=True) #Заменим NaN значения на некое новое значение атрибута (надо еще убедиться что такого нет...)
Xcat = Xcat.applymap(lambda x:str(x))
Xcat = pd.DataFrame(Xcat)

dv = fe.DictVectorizer(sparse = False)
cat_encoded = dv.fit_transform(Xcat.T.to_dict().values())

print cat_encoded.shape #если не убирать признаки с большим количеством уникальных значений, то получается 50713 столбцов

C:\Users\Anton\Anaconda2\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


(26800L, 316L)


__Нормируем числовые признаки и заменим значения NaN__

In [12]:
#нормируем данные, чтобы гдадиент точнее указывал на локальный минимум ошибки 
#(см. 3-2.Prakticheskie_rekomendacii_po_linejnym_modelyam.pdf)
data_num_norm = data[num_columns].copy()
for x in data_num_norm.columns:
    data_num_norm[x] = (data[x] - data[x].dropna().mean())/data[x].dropna().std()

#заполним неопределенные значения NaN средними по признаку
for x in data_num_norm[num_columns]:
    data_num_norm[x].fillna(data_num_norm[x].dropna().mean(), inplace=True)

data_num_norm = pd.concat((data_num_norm, data.Class), axis=1)
data_num_norm.head(1)

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190,Class
0,1.016715e-16,-0.038208,-0.159295,7.121659e-17,-7.401487e-17,-3.894898e-17,-5.709958e-16,-9.053958e-18,-3.585545e-17,-0.214707,...,-9.322133e-16,-1.874740e-16,-0.361547,-0.18284,-1.445665e-16,1.261617e-17,-1.284743,1.037569,-1.684699e-16,-1


__Объединим значения числовых и закодированных категориальных признаков в общий набор данных__

In [13]:
#именуем столбцы закодированных категориальных признаков
cat_enc_columns = ['Cat'+ str(x) for x in range(0,cat_encoded.shape[1])] 
#и создаем для них DataFrame
cat_enc_df = pd.DataFrame(cat_encoded, columns = cat_enc_columns)
#объединяем числовые и категориальные признаки
X = pd.concat((data_num_norm.drop('Class', axis=1),cat_enc_df), axis=1)
y = data_num_norm.Class

X.head(1)

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Cat306,Cat307,Cat308,Cat309,Cat310,Cat311,Cat312,Cat313,Cat314,Cat315
0,1.016715e-16,-0.038208,-0.159295,7.121659e-17,-7.401487e-17,-3.894898e-17,-5.709958e-16,-9.053958e-18,-3.585545e-17,-0.214707,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


###  Построение моделей

Для оценки моделей будем использовать кросс-валидацию по методу StratifiedKFold

In [21]:
#skf = StratifiedKFold(n_splits = 3, shuffle = False, random_state = 0)
skf = cross_validation.StratifiedKFold(y, n_folds = 3, shuffle = True, random_state = 0)

### 1. Линейная модель

In [22]:
lm_ridge = lm.RidgeClassifier(class_weight='balanced') #к сожалению, без balanced количество прогноза оттока всего 4 объекта
lm_ridge.fit(X,y)
print 'Предсказано %i объектов оттока' % len([x for x in lm_ridge.predict(X) if x == 1])

Предсказано 10687 объектов оттока


In [23]:
for s in [scorer.roc_auc_scorer, scorer.f1_scorer,  scorer.precision_scorer,  scorer.recall_scorer]:
    print s, cross_val_score(estimator = lm_ridge, X = X, y = y, scoring = s, cv = skf).mean()
    

make_scorer(roc_auc_score, needs_threshold=True) 0.6625263844298529
make_scorer(f1_score) 0.19503050238960692
make_scorer(precision_score) 0.11591664133345563
make_scorer(recall_score) 0.6143438415315398


_Для сравнения, только числовые признаки:_   
make_scorer(roc_auc_score, needs_threshold=True) 0.4925389510642966   
make_scorer(f1_score) 0.08059739700603331   
make_scorer(precision_score) 0.04766081871345029   
make_scorer(recall_score) 0.2619047619047619   

### 2. Случайный лес

In [24]:
rfc = es.RandomForestClassifier(random_state=0, class_weight='balanced')

In [25]:
for s in [scorer.roc_auc_scorer, scorer.f1_scorer,  scorer.precision_scorer,  scorer.recall_scorer]:
    print s, cross_val_score(estimator = rfc, X = X, y = y, scoring = s, cv = skf).mean()

make_scorer(roc_auc_score, needs_threshold=True) 0.5869036321978011
make_scorer(f1_score) 0.006934318117738318
make_scorer(precision_score) 0.3452380952380952
make_scorer(recall_score) 0.0035102817284174293


_Для сравнения, только числовые признаки:_    
make_scorer(roc_auc_score, needs_threshold=True) 0.4544656572306342   
make_scorer(f1_score) 0.0   
make_scorer(precision_score) 0.0   
make_scorer(recall_score) 0.0   

### 3. Градиентный бустинг

In [26]:
gbc =  es.GradientBoostingClassifier(random_state=0)

In [27]:
for s in [scorer.roc_auc_scorer, scorer.f1_scorer,  scorer.precision_scorer,  scorer.recall_scorer]:
    print s, cross_val_score(estimator = gbc, X = X, y = y, scoring = s, cv = skf).mean()

make_scorer(roc_auc_score, needs_threshold=True) 0.7310207951597607
make_scorer(f1_score) 0.019496042186357802
make_scorer(precision_score) 0.4101358411703239
make_scorer(recall_score) 0.010028837153123773


_только числовые признаки:_   
make_scorer(roc_auc_score, needs_threshold=True) 0.5071660815617827   
make_scorer(f1_score) 0.0   
make_scorer(precision_score) 0.0   
make_scorer(recall_score) 0.0   

### Выводы

__Линейный классификатор:__   
make_scorer(roc_auc_score, needs_threshold=True) 0.6703990960152332   
__make_scorer(f1_score) 0.19992168247893316__   
make_scorer(precision_score) 0.11890759404800112   
__make_scorer(recall_score) 0.6273584764503976__   

__Случайный лес:__   
make_scorer(roc_auc_score, needs_threshold=True) 0.5963273579911893   
make_scorer(f1_score) 0.0039375068283622845   
make_scorer(precision_score) 0.14166666666666666   
make_scorer(recall_score) 0.001996756112961483   

__Градиентный бустинг:__   
__make_scorer(roc_auc_score, needs_threshold=True) 0.7358515228926609__   
make_scorer(f1_score) 0.026930388542004885   
__make_scorer(precision_score) 0.4046474358974359__   
make_scorer(recall_score) 0.013988514724673592   

Среди классификаторов борьба идет только между линейным классификатором и градиентным бустингом. Случайный лес проигрывыает по всем параметрам. Как мы говорили, основной метрикой является ROC-AUC, а по ней с большим отрывом лидирует градиентный бустинг.
Конечно, линейный классификатор выигрывает по полноте (recall), но при такой точности (prescision 0.11) вряд ли это реальное преимущество. Наоборот, градиентный бустинг показывает намного лучшую точность, но небольшую полноту.
Все же, честно говоря, при таких абсолютных значениях (да можно даже по f-мере посмотреть), говорить о практической значимости результатов не приходится. Однозначно, надо заниматься оптимизацией модели, чтобы выбирать, лучшую модель из хороших, а не из плохих.


Пока же вывод простой: побеждает градиейнтный бустинг, на втором месте линейный классификатор, на третьем случайный лес.